## Name Scopes

In [1]:
import tensorflow as tf
import numpy as np

C:\Anaconda\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
graph = tf.Graph()

In [4]:
with graph.as_default():
    with tf.name_scope("Variables"):
        global_step =tf.Variable(0,dtype=tf.int32,trainable=False,name="global_step")
        total_output=tf.Variable(0.0,dtype=tf.float32,trainable=False,name="total_output")
        
    with tf.name_scope("transformation"):
        with tf.name_scope("input"):
            a=tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")
            
        with tf.name_scope("intermediate_layer"):
            b=tf.reduce_prod(a,name="product_b")
            c=tf.reduce_sum(a,name="sum_c")
        
        with tf.name_scope("output"):
            output=tf.add(b,c,name="output")
    with tf.name_scope("update"):
        #Incrementsthetotal_output Variable bythe latestoutput
        update_total=total_output.assign_add(output)
        # Increments this variable for every run of graph
        increment_step = global_step.assign_add(1)
    with tf.name_scope("summaries"):
        avg=tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
        #Creates summaries for output node
        tf.summary.scalar('Output',output)
        tf.summary.scalar("total_summary",update_total)
        tf.summary.scalar('Average_Summary',avg)
    with tf.name_scope('ops'):
        init = tf.global_variables_initializer()
        merged_summaries = tf.summary.merge_all()


In [5]:
sess = tf.Session(graph=graph)
writer=tf.summary.FileWriter('./graphv1',	graph=graph)

In [6]:
sess.run(init)

In [7]:
init

<tf.Operation 'ops/init' type=NoOp>

In [8]:
def run_graph(input_tensor):
    input_tensor = np.asarray(input_tensor)
    print (input_tensor)
    feed_dict ={a:input_tensor}
    print (feed_dict)
    _,step,summary=sess.run([output,increment_step,merged_summaries],feed_dict=feed_dict)
    writer.add_summary(summary,global_step=step)

In [9]:
run_graph([2.0,3.0])
run_graph([3,4])
run_graph([2,2])
run_graph([2,3])


[2. 3.]
{<tf.Tensor 'transformation/input/input_placeholder_a:0' shape=(?,) dtype=float32>: array([2., 3.])}
[3 4]
{<tf.Tensor 'transformation/input/input_placeholder_a:0' shape=(?,) dtype=float32>: array([3, 4])}
[2 2]
{<tf.Tensor 'transformation/input/input_placeholder_a:0' shape=(?,) dtype=float32>: array([2, 2])}
[2 3]
{<tf.Tensor 'transformation/input/input_placeholder_a:0' shape=(?,) dtype=float32>: array([2, 3])}


In [10]:
writer.flush()
writer.close()
sess.close()